In [1]:
import os
os.chdir("/Users/adityachhabra/Github/zavmo/zavmo-api/zavmo")

In [7]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import Markdown

In [8]:
from helpers.agents.common import get_yaml_data,get_agent_instructions

In [26]:
from pydantic import BaseModel, Field, validator
from typing import Literal, List, Optional, Dict
from helpers.swarm import Agent, Response, Result, Tool, with_context
from helpers.agents.common import get_agent_instructions

In [27]:
### For handoff
def transfer_to_discussion_agent():
    """Transfer to the Discussion Agent when the learner is satisfied with the summary of the information gathered."""
    print("Transferring to Discussion Agent...")
    return None

### For updating the data

class update_discover_data(Tool):
    """Update the learner's information gathered during the Discovery stage."""
    learning_goals: str = Field(description="The learner's learning goals.")
    learning_goal_rationale: str = Field(description="The learner's rationale for their learning goals.")
    knowledge_level: Literal["Beginner", "Intermediate", "Advanced", "Expert"] = Field(description="The learner's self-assessed knowledge level in their chosen area of study.")
    application_area: str = Field(description="A specific area or context where the learner plans to apply their new knowledge and skills.")
    
    def __str__(self):
        """Return a string representation of the UpdateDiscoverData object."""
        string = []
        for field, value in self.__dict__.items():
            string.append(f"{field}: {value}")
        return "\n".join(string)
    
    @with_context
    def execute(self, context: Dict):
        # Get email and sequence_id from context
        email       = context.get('email')
        sequence_id = context.get('sequence_id')
        if not email:
            raise ValueError("Email not found in context")
        if not sequence_id:
            raise ValueError("Sequence ID not found in context")
        
        # # Get the DiscoverStage object
        # discover_stage = DiscoverStage.objects.get(user_email=email, sequence_id=sequence_id)
        # if not discover_stage:
        #     raise ValueError("DiscoverStage not found")
        
        # # Update the DiscoverStage object
        # discover_stage.learning_goals = self.learning_goals
        # discover_stage.learning_goal_rationale = self.learning_goal_rationale
        # discover_stage.knowledge_level = self.knowledge_level
        # discover_stage.application_area = self.application_area
        # discover_stage.save()
        
        return Result(value=f"Updated DiscoverStage for {email} with sequence ID {sequence_id}. The following fields were updated: {str(self)}")
            

In [25]:
#### For responding / handoff
discover_agent = Agent(
    name="Discover",
    model="gpt-4o-mini",
    instructions=get_agent_instructions('discover'),
    functions=[
        update_discover_data,
        transfer_to_discussion_agent
    ],
    tool_choice="auto",
    parallel_tool_calls=False
)


In [33]:
agent = discover_agent
messages = []
context = {"email":"aditya@gmail.com","sequence_id":'52442'}

In [97]:
result = run_step(discover_agent, messages, context=context)

2024-10-24 15:50:06,523 - INFO - Running step 0 with agent Discover
2024-10-24 15:50:10,275 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Input tokens: 1470
Output tokens: 267
Total tokens: 1737


In [98]:
messages.extend(result.messages)

In [99]:
Markdown(messages[-1]['content'])

Awesome! 🎉 Based on your interests in all areas of business, your hands-on learning style, and your availability of 8 hours each week over the next 10 weeks, we’ve crafted a dynamic curriculum just for you. Here’s a quick overview:

## Comprehensive Business Curriculum
| **Subject** | Various Business Topics | **Level** | Intermediate |
### Modules
| Module | Duration | Learning Outcomes | Hands-on Activities |
|--------|----------|-------------------|--------------------|
| **Business Strategy** | 2 weeks | Understand market positioning & competitive analysis | SWOT analysis workshop, strategy simulation |
| **Marketing** | 2 weeks | Develop effective marketing strategies | Create a marketing campaign project |
| **Financial Management** | 2 weeks | Manage budgets and financial statements | Budgeting exercise with real case studies |
| **Operations Management** | 2 weeks | Learn about process optimization | Design an operations improvement project |
| **Entrepreneurship** | 2 weeks | Skills for starting and managing a business | Create a business plan for a startup idea |

This plan will give you a comprehensive understanding of each topic while allowing you to apply what you learn in practical ways! 📈💡

What do you think of this proposed curriculum? Does it align with your learning goals and fit your available time? 😊

In [95]:
messages.append({"role":"user","content":"yes it does"})

In [96]:
messages

[{'content': "Perfect, Aditya! With 8 hours per week to dedicate, we can create a well-structured curriculum that balances depth and practical application. \n\nNow, let's discuss your specific interests in the field of business. What areas are you most passionate about exploring? 🎯",
  'refusal': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': None,
  'sender': 'Discover'},
 {'role': 'user', 'content': 'everything'},
 {'content': "That's awesome, Aditya! 🌟 Since you're open to exploring everything in business, we can cover a range of topics. Here are some key areas we could focus on:\n\n1. **Business Strategy**: Understanding how to position your business in the market.\n2. **Marketing**: Learning how to reach and engage customers.\n3. **Financial Management**: Getting insights into managing finances and budgets.\n4. **Operations Management**: Streamlining business processes for efficiency.\n5. **Entrepreneurship**: Skills for starting and managing your own ventur